# Import Libraries

In [1]:
#pip install vaderSentiment

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
import spacy
import re
import nltk
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words -= {"not", "do"}
from autocorrect import Speller
nltk.download('words', quiet=True)
import emoji
import sqlite3
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.corpus import words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import contractions
nltk.download('wordnet', quiet=True)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from pprint import pprint
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm


# Import dataset

In [4]:
conn = sqlite3.connect("IMDB_Movies_2021.db")

cursor = conn.cursor()

table_name = "REVIEWS"

df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

conn.close()

# Drop duplicates and remove nulls

In [5]:
df = df.drop_duplicates(subset = ['REVIEW','AUTHOR'])
df = df.dropna()

# Combine review and title

In [6]:
df['FIXED_REVIEW'] = df['REVIEW'] +df['TITLE']

In [7]:
df['FIXED_REVIEW']

0       I don't get all the terrible reviews for this ...
1       I cannot believe anyone could give this film l...
2       Great White is not the worst way to spend 90 m...
3       Great White is as basic of a killer shark film...
4       Terrible story, dialogue and CGI. The film has...
                              ...                        
5420    It's master piece by Zack please part 2,3,4 al...
5421    No words to describe. It's awesome. One of the...
5422    Far better than previous one and better editin...
5423    Why did the studio say no to this masterpiece?...
5424    Overall Opinion-\nAlthough the competitors Mar...
Name: FIXED_REVIEW, Length: 5304, dtype: object

# Simple preprocesing: lowercase, punctuation, emojis

In [8]:
# Cleaned the text by using Regex
df['FIXED_REVIEW'] = [review.lower() for review in df['FIXED_REVIEW']]   # Lowercase
df['FIXED_REVIEW'] = [re.sub(r'[^a-zA-Z\s]', '', review) for review in df['FIXED_REVIEW']]  # Remove punctuation and numbers
df['FIXED_REVIEW'] = [re.sub(r'@\w+', '', review) for review in df['FIXED_REVIEW']]   # Remove mentions
df['FIXED_REVIEW'] = [re.sub(r'#\w+', '', review) for review in df['FIXED_REVIEW']]   # Remove hashtags
df['FIXED_REVIEW'] = [re.sub(r'http\S+|www\S+|https\S+', '', review) for review in df['FIXED_REVIEW']]  # Remove URLS
df['FIXED_REVIEW'] = [emoji.demojize(review) for review in df['FIXED_REVIEW']]    # Emoji to text

In [9]:
df['FIXED_REVIEW']

0       i dont get all the terrible reviews for this m...
1       i cannot believe anyone could give this film l...
2       great white is not the worst way to spend  min...
3       great white is as basic of a killer shark film...
4       terrible story dialogue and cgi the film has a...
                              ...                        
5420    its master piece by zack please part  also man...
5421    no words to describe its awesome one of the be...
5422    far better than previous one and better editin...
5423    why did the studio say no to this masterpiece ...
5424    overall opinion\nalthough the competitors marv...
Name: FIXED_REVIEW, Length: 5304, dtype: object

# Remove stop short and long words, lemmatize

In [10]:
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('no')
def sentiment(word):
    blob = TextBlob(word)
    sentiment = blob.sentiment.polarity
    return np.abs(sentiment)

In [11]:
def remove_stop_words(text):
    text = contractions.fix(text)
    words = text.split()    # Tokenize the text into words
    lemmatizer = WordNetLemmatizer()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    filtered_words = [word for word in filtered_words if len(word) >= 2]
    filtered_words = [word for word in filtered_words if len(word) < 10]
    filtered_words = [word for word in filtered_words if sentiment(word) != 0 ]
    return ' '.join(filtered_words)

In [12]:
df['FIXED_REVIEW'] = df['FIXED_REVIEW'].apply(remove_stop_words)

In [13]:
df['FIXED_REVIEW']

0       terrible bad special worthy terrible good litt...
1                                 less great exciting bad
2       great worst wealthy victim great cheap great c...
3       great everyday ordinary ok certain real bloody...
4                  terrible wide first much much terrible
                              ...                        
5420                                                     
5421                               awesome best fantastic
5422                   far better previous better awesome
5423                                  seriously mean epic
5424    comic enjoy clear perfect strong previous clea...
Name: FIXED_REVIEW, Length: 5304, dtype: object

# Remove misspelt words (maybe replace with better method)

In [14]:
spell = Speller()
df['FIXED_REVIEW'] = df['FIXED_REVIEW'].apply(spell)

In [15]:
setofwords = set(words.words())

print("excited" in setofwords)

True


In [16]:
j = 0
new_review = []

for i in df['FIXED_REVIEW']:
    words = word_tokenize(i)
    new_words = []
    for word in words:
        if word in setofwords:
            new_words.append(word)
    sentence_correct = ' '.join(new_words)        
    new_review.append(sentence_correct)
    #df['FIXED_REVIEW'][j] = sentence_correct
    j+=1


df['FIXED_REVIEW'] = new_review

In [17]:
print(df['FIXED_REVIEW'].head())

0    terrible bad special worthy terrible good litt...
1                              less great exciting bad
2    great worst wealthy victim great cheap great c...
3    great everyday ordinary certain real bloody ki...
4               terrible wide first much much terrible
Name: FIXED_REVIEW, dtype: object


# Add our target (ie: class)

In [18]:
def label_race(row):
    if row['RATING'] >= 7:
        return 2
    if row['RATING'] <= 3:
        return 0
    else:
        return 1

df['CLASS'] = df.apply(label_race, axis=1) 

In [19]:
print(df['CLASS'].value_counts())

2    2191
0    2014
1    1099
Name: CLASS, dtype: int64


# Add sentiment as column (can add more)

In [20]:
# Calculate Sentiment Score with TextBlob
def get_sentiment_1(text):
    return TextBlob(text).sentiment.polarity 
df['sentiment_1'] = df['FIXED_REVIEW'].apply(get_sentiment_1)

In [21]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    return compound_score

def vader_class(text):
    sentiment_score = analyzer.polarity_scores(text)
    compound_score = sentiment_score['compound']
    
    if compound_score > 0.1:
        sentiment = 2
    elif compound_score < -0.1:
        sentiment = 0
    else:
        sentiment = 1
    
    return sentiment

In [22]:
df['vader_sentiment'] = df['FIXED_REVIEW'].apply(vader_sentiment)
df['vader_class'] = df['FIXED_REVIEW'].apply(vader_class)

In [23]:
print(df[['sentiment_1', 'vader_sentiment', 'vader_class']].head())

   sentiment_1  vader_sentiment  vader_class
0    -0.299669          -0.8036            0
1     0.058333           0.5412            2
2     0.321807           0.9839            2
3    -0.021337           0.9133            2
4    -0.462500          -0.7351            0


# Add a tokenized column for topics

In [24]:
df['tokenized_review'] = df['FIXED_REVIEW'].apply(word_tokenize)

In [25]:
print(df['tokenized_review'].head())

0    [terrible, bad, special, worthy, terrible, goo...
1                         [less, great, exciting, bad]
2    [great, worst, wealthy, victim, great, cheap, ...
3    [great, everyday, ordinary, certain, real, blo...
4        [terrible, wide, first, much, much, terrible]
Name: tokenized_review, dtype: object


# Now use LDA for topics, gives additional context (may need to optimise how many topics

In [26]:
dictionary = corpora.Dictionary(df['tokenized_review'])
corpus = [dictionary.doc2bow(text) for text in df['tokenized_review']]

In [27]:
num_topics=20
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=20, alpha="auto")

In [28]:
topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]

In [29]:
topic_matrix = []
for dist in topic_distributions:
    topic_vector = [0] * num_topics  # Number of topics (in this case 2)
    for topic_id, prob in dist:
        topic_vector[topic_id] = prob
    topic_matrix.append(topic_vector)

# Convert topic_matrix to a DataFrame for easy handling
topic_df = pd.DataFrame(topic_matrix, columns=[f"topic_{i+1}" for i in range(num_topics)])

In [30]:
df = pd.concat([df, topic_df], axis=1)
df = df.dropna()

In [31]:
df.head()

,ID,REVIEW,RATING,AUTHOR,TITLE,FIXED_REVIEW,CLASS,sentiment_1,vader_sentiment,vader_class,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,1.0,I don't get all the terrible reviews for this ...,5.0,margarida-44311,Not Bad\n,terrible bad special worthy terrible good litt...,1.0,-0.299669,-0.8036,0.0,...,0.000000,0.000000,0.476151,0.000000,0.000000,0.000000,0.000000,0.143583,0.000000,0.000000
1,2.0,I cannot believe anyone could give this film l...,8.0,joemay-2,What are all the bad reviews about is it a wo...,less great exciting bad,2.0,0.058333,0.5412,2.0,...,0.011021,0.013186,0.203828,0.012336,0.013305,0.016907,0.375350,0.012423,0.014082,0.015123
2,3.0,Great White is not the worst way to spend 90 m...,4.0,nebk,Great White=Jaws Lite\n,great worst wealthy victim great cheap great c...,1.0,0.321807,0.9839,2.0,...,0.000000,0.055826,0.033114,0.000000,0.000000,0.112409,0.304684,0.000000,0.057025,0.000000
3,4.0,Great White is as basic of a killer shark film...,4.0,kuarinofu,Bare-bones killer shark film\n,great everyday ordinary certain real bloody ki...,1.0,-0.021337,0.9133,2.0,...,0.000000,0.000000,0.010425,0.000000,0.096944,0.110776,0.000000,0.000000,0.000000,0.207216
4,5.0,"Terrible story, dialogue and CGI. The film has...",4.0,Horror_Flick_Fanatic,"Terrible story, dialogue, and CGI\n",terrible wide first much much terrible,1.0,-0.462500,-0.7351,0.0,...,0.000000,0.000000,0.282749,0.000000,0.000000,0.012546,0.336880,0.000000,0.010451,0.011222


# Now BOW with tf idf

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)

In [33]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['FIXED_REVIEW'])

tfidf_df = pd.DataFrame.sparse.from_spmatrix(
    tfidf_matrix,
    columns=tfidf_vectorizer.get_feature_names_out()
)

In [34]:
df = pd.concat([df.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

In [35]:
df = df.drop(columns=['ID', 'RATING', 'REVIEW', 'TITLE'], errors='ignore')

In [36]:
df = df.dropna()

In [37]:
df.shape

(5187, 5027)

# Save data

In [38]:
#df.to_csv('preprocessed_data.csv')

# Using the data in models

In [39]:
X = df.drop(columns = ['CLASS', 'AUTHOR','FIXED_REVIEW','tokenized_review'])
y = df['CLASS']

# Feature Selection: RFE Method

In [40]:
rf = RandomForestClassifier(random_state=27, n_jobs=-1)
rf.fit(X, y)

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

num_feature = 2000
top_features_rf = [X.columns[i] for i in indices[:num_feature]]
print(f"Selected Features (Random Forest): {top_features_rf}")

X_selected = X[top_features_rf]

Selected Features (Random Forest): ['sentiment_1', 'vader_sentiment', 'vader_class', 'topic_13', 'topic_10', 'topic_6', 'topic_17', 'topic_2', 'topic_16', 'waste', 'topic_19', 'topic_1', 'topic_20', 'topic_12', 'topic_18', 'great', 'topic_3', 'topic_15', 'topic_14', 'topic_11', 'topic_7', 'good', 'bad', 'topic_4', 'topic_5', 'topic_8', 'boring', 'really', 'topic_9', 'terrible', 'worst', 'better', 'much', 'amazing', 'love', 'perfect', 'awful', 'funny', 'best', 'little', 'real', 'horrible', 'beautiful', 'new', 'first', 'action', 'poor', 'many', 'fun', 'kind', 'pretty', 'decent', 'worth', 'long', 'half', 'excellent', 'main', 'mean', 'whole', 'slow', 'average', 'black', 'mess', 'sure', 'nice', 'stupid', 'enjoy', 'fly', 'worse', 'young', 'brilliant', 'less', 'original', 'enjoyable', 'old', 'right', 'hard', 'high', 'wonderful', 'laugh', 'wasted', 'poorly', 'least', 'wrong', 'true', 'far', 'fine', 'dull', 'fantastic', 'weak', 'pointless', 'really good', 'dumb', 'mediocre', 'powerful', 'highly

In [41]:
X_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5187 entries, 0 to 5186
Columns: 2000 entries, sentiment_1 to less least
dtypes: Sparse[float64, 0](1977), float64(23)
memory usage: 1.5 MB


# PCA: Dimensionality Reduction

## Again need to find optimum k

In [42]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Standardize the data
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_selected)

In [43]:
n_components=100
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# Convert to DataFrame
X_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])

In [44]:
X_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5187 entries, 0 to 5186
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     5187 non-null   float64
 1   PC2     5187 non-null   float64
 2   PC3     5187 non-null   float64
 3   PC4     5187 non-null   float64
 4   PC5     5187 non-null   float64
 5   PC6     5187 non-null   float64
 6   PC7     5187 non-null   float64
 7   PC8     5187 non-null   float64
 8   PC9     5187 non-null   float64
 9   PC10    5187 non-null   float64
 10  PC11    5187 non-null   float64
 11  PC12    5187 non-null   float64
 12  PC13    5187 non-null   float64
 13  PC14    5187 non-null   float64
 14  PC15    5187 non-null   float64
 15  PC16    5187 non-null   float64
 16  PC17    5187 non-null   float64
 17  PC18    5187 non-null   float64
 18  PC19    5187 non-null   float64
 19  PC20    5187 non-null   float64
 20  PC21    5187 non-null   float64
 21  PC22    5187 non-null   float64
 22 

# LASSO TBC

# Now onto our models

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=27, stratify=y)

# Multinomial Logistic Regression (MLR)

## MLR - Initial Run

In [46]:
model_mlr = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                                   class_weight='balanced', max_iter=1000)

model_mlr.fit(X_train, y_train)

y_pred = model_mlr.predict(X_test)
print("Initial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))

Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.72      0.72      0.72       398
         1.0       0.40      0.44      0.42       214
         2.0       0.76      0.73      0.74       426

    accuracy                           0.67      1038
   macro avg       0.63      0.63      0.63      1038
weighted avg       0.67      0.67      0.67      1038



## MLR- Hyperparameter Tuning

In [47]:
param_grid = {
    'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'saga'],
}

grid = GridSearchCV(
    LogisticRegression(multi_class='multinomial', class_weight='balanced', max_iter=1000),
    param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("\nBest Parameters:", grid.best_params_)

y_pred_tuned = grid.predict(X_test)
print("\nTuned Classification Report:")
print(classification_report(y_test, y_pred_tuned, zero_division=1))

<IPython.core.display.Javascript object>


Best Parameters: {'C': 1e-05, 'solver': 'saga'}

Tuned Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.73      0.72       398
         1.0       0.45      0.35      0.39       214
         2.0       0.72      0.79      0.76       426

    accuracy                           0.67      1038
   macro avg       0.63      0.62      0.62      1038
weighted avg       0.66      0.67      0.67      1038



# Random Forest (RF)

## RF - Initial Run

In [48]:
model_rf = RandomForestClassifier(random_state=27, class_weight='balanced')

model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

print("\nInitial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))


Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.77      0.73       398
         1.0       0.51      0.17      0.25       214
         2.0       0.69      0.85      0.76       426

    accuracy                           0.68      1038
   macro avg       0.63      0.59      0.58      1038
weighted avg       0.65      0.68      0.64      1038



## RF - Hyperparameter Tuning

In [49]:
param_dist = {
    'n_estimators': randint(50, 1000),
    'max_depth': randint(2, 50),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

rand_search = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=27
)

rand_search.fit(X_train, y_train)

best_rf = rand_search.best_estimator_

y_pred_best = best_rf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

print("\nBest Hyperparameters:", rand_search.best_params_)

print("\nTuned RF Classification Report:")
print(classification_report(y_test, y_pred_best, zero_division=1))


Best Hyperparameters: {'bootstrap': True, 'max_depth': 27, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 689}

Tuned RF Classification Report:
              precision    recall  f1-score   support

         0.0       0.73      0.77      0.75       398
         1.0       0.55      0.29      0.38       214
         2.0       0.70      0.83      0.76       426

    accuracy                           0.70      1038
   macro avg       0.66      0.63      0.63      1038
weighted avg       0.68      0.70      0.68      1038



# Support Vector Machine (SVM)

## SVM- Initial Run

In [50]:
model_svm = svm.SVC(kernel="linear", random_state=27, class_weight='balanced')

model_svm.fit(X_train, y_train)
y_pred = model_svm.predict(X_test)

print("\nInitial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))


Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.75      0.73       398
         1.0       0.39      0.38      0.38       214
         2.0       0.76      0.73      0.75       426

    accuracy                           0.67      1038
   macro avg       0.62      0.62      0.62      1038
weighted avg       0.67      0.67      0.67      1038



## SVM - Hyperparameter Tuning

In [51]:
param_grid = {
    'C': [0.1, 1, 5],
    'kernel': ['linear', 'rbf'],
}

grid = GridSearchCV(
    estimator=model_svm,
    param_grid=param_grid,
    scoring="accuracy",
    n_jobs=-1,
    cv=5,
    verbose=3
)

grid.fit(X_train, y_train)

print("\nBest Parameters:", grid.best_params_)

y_pred = grid.predict(X_test)
print("\nTuned Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))

<IPython.core.display.Javascript object>

Fitting 5 folds for each of 6 candidates, totalling 30 fits

Best Parameters: {'C': 5, 'kernel': 'rbf'}

Tuned Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.75      0.73       398
         1.0       0.46      0.40      0.43       214
         2.0       0.75      0.75      0.75       426

    accuracy                           0.68      1038
   macro avg       0.64      0.63      0.64      1038
weighted avg       0.67      0.68      0.68      1038



# XGBoost (XGB)

In [52]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss', use_label_encoder=False)
model_xgb.fit(X_train, y_train)

y_pred = model_xgb.predict(X_test)
print("\nInitial Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))


Initial Classification Report:
              precision    recall  f1-score   support

         0.0       0.72      0.80      0.76       398
         1.0       0.49      0.27      0.34       214
         2.0       0.72      0.81      0.76       426

    accuracy                           0.69      1038
   macro avg       0.64      0.62      0.62      1038
weighted avg       0.67      0.69      0.67      1038



In [53]:
from scipy.stats import randint, uniform

param_dist = {
    'n_estimators': randint(50, 500),       
    'max_depth': randint(3, 15),      
    'learning_rate': uniform(0.01, 0.3),   
    'subsample': uniform(0.5, 0.5),      
    'colsample_bytree': uniform(0.5, 0.5), 
    'gamma': uniform(0, 5),       
    'reg_alpha': uniform(0, 1),  
    'reg_lambda': uniform(1, 5)   
}

rand_search = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_dist,
    n_iter=5,              
    cv=5,                
    scoring='accuracy',  
    n_jobs=-1,      
    random_state=27,
    verbose=3
)

rand_search.fit(X_train, y_train)

best_xgb = rand_search.best_estimator_
y_pred_best = best_xgb.predict(X_test)

print("\nBest Hyperparameters:", rand_search.best_params_)
print("\nTuned Classification Report:")
print(classification_report(y_test, y_pred_best, zero_division=1))

Fitting 5 folds for each of 5 candidates, totalling 25 fits

Best Hyperparameters: {'colsample_bytree': 0.500728347620244, 'gamma': 0.3369354595593149, 'learning_rate': 0.07172079621340263, 'max_depth': 9, 'n_estimators': 254, 'reg_alpha': 0.16981215156572083, 'reg_lambda': 5.775213080688159, 'subsample': 0.6197745550616657}

Tuned Classification Report:
              precision    recall  f1-score   support

         0.0       0.72      0.81      0.76       398
         1.0       0.53      0.27      0.36       214
         2.0       0.73      0.84      0.78       426

    accuracy                           0.71      1038
   macro avg       0.66      0.64      0.63      1038
weighted avg       0.69      0.71      0.69      1038

